In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [210]:
from torchvision import transforms

def getTransform():
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(
            mean=[0.485, 0.456, 0.406],
            std=[0.229, 0.224, 0.225]
        )
    ])
    return transform


In [211]:
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

def loadImages(imageFolder, batchSize=32):


    transform = getTransform()

    dataset = ImageFolder(root=imageFolder, transform=transform)
    loader = DataLoader(dataset, batch_size=batchSize, shuffle=False)

    sampleImg, _ = dataset[0]
    print("Sample image shape after transform:", sampleImg.shape)

    return dataset, loader

In [212]:
from torchvision.models import resnet18
from torchvision.models.feature_extraction import create_feature_extractor
import torch

def getResnet18FeatureExtractor():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = resnet18(pretrained=True).to(device)
    model.eval()

    # Extract last convolutional layer (last conv of layer4)
    returnNodes = {'layer4.1.conv2': 'features'}
    featureExtractor = create_feature_extractor(model, return_nodes=returnNodes)

    return featureExtractor, device


In [213]:
 #Passes images in the loader through the feature extractor and
 #returns flattened features and labels.

def extractFeatures(loader, featureExtractor, device):

    allFeatures = []
    allLabels = []

    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            features = featureExtractor(images)['features']
            features = features.view(features.size(0), -1)  # flatten H x W x C
            allFeatures.append(features.cpu())
            allLabels.append(labels)

    allFeatures = torch.cat(allFeatures)
    allLabels = torch.cat(allLabels)

    return allFeatures, allLabels


In [214]:
from sklearn.decomposition import PCA

def reduceTo2D(features):

    pca = PCA(n_components=2)
    reduced = pca.fit_transform(features)

    print("Original feature dim:", features.shape[1])
    print("Reduced feature dim:", reduced.shape[1])

    return reduced


In [215]:
from sklearn.cluster import KMeans, BisectingKMeans

def kmeansApproach(features2D, K=3):
    results = {}


    kmRandom = KMeans(n_clusters=K, init='random', n_init=10)
    results["KMeans_random"] = kmRandom.fit_predict(features2D)


    kmPlus = KMeans(n_clusters=K, init='k-means++', n_init=10)
    results["KMeans_kmeans++"] = kmPlus.fit_predict(features2D)


    kmBisect = BisectingKMeans(n_clusters=K, init='random')
    results["Bisecting_KMeans"] = kmBisect.fit_predict(features2D)

    return results


In [216]:
from sklearn.metrics import fowlkes_mallows_score, silhouette_score


def evaluateClustering(predLabels, trueLabels=None, features=None, method="fowlkes"):

    method = method.lower()

    if method == "fowlkes":
        return fowlkes_mallows_score(trueLabels, predLabels)

    elif method == "silhouette":
        return silhouette_score(features, predLabels)

    else:
        return None


In [217]:
from sklearn.cluster import SpectralClustering

def spectralClusteringApproacha(X):
    spectral = SpectralClustering(n_clusters=3, assign_labels='kmeans')
    labels = spectral.fit_predict(X)
    return labels




In [218]:
from sklearn.metrics.pairwise import rbf_kernel
#First one was not producing desired clustering

def spectralClusteringApproach(features2D):
    # Use RBF kernel to compute similarity (default gamma=None uses 1/n_features)
    affinity_matrix = rbf_kernel(features2D)

    # Spectral Clustering with precomputed affinity to better separate clusters
    spectral = SpectralClustering(n_clusters=3, affinity='precomputed', assign_labels='kmeans', random_state=42)
    labels = spectral.fit_predict(affinity_matrix)
    return labels


In [219]:
from sklearn.cluster import AgglomerativeClustering

def agglomerativeClusteringAll(features2D, n_clusters=3):

    linkageMethods = ['single', 'complete', 'average', 'ward']
    labelsDict = {}

    for linkage in linkageMethods:
        agglo = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
        labels = agglo.fit_predict(features2D)
        labelsDict[linkage] = labels

    return labelsDict


In [220]:
def dbscanApproach(pca2Dfeatures, neighborhoodRadius, minPointsInCluster):
    from sklearn.cluster import DBSCAN

    dbscanModel = DBSCAN(eps=neighborhoodRadius, min_samples=minPointsInCluster)
    db = dbscanModel.fit(pca2Dfeatures)


    labels = db.labels_
    return labels


In [221]:
def testDbscanPair(features2D, eps, minSamples):
    labels = dbscanApproach(features2D, eps, minSamples)

    if labels is None or len(labels) == 0:
        return 0, []


    uniqueLabels = set(labels)
    if -1 in uniqueLabels:
        clusterCount = len(uniqueLabels) - 1
    else:
        clusterCount = len(uniqueLabels)

    return clusterCount, labels



In [222]:
def searchClusterSize(features2D, desired_clusters=3):
    #Search for DBSCAN parameters that produce the desired number of clusters.

    eps_values = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
    min_samples_values = [3, 4, 5, 6, 7, 8]

    for eps in eps_values:
        for minSamples in min_samples_values:
            clusterCount, labels = testDbscanPair(features2D, eps, minSamples)

            if clusterCount == desired_clusters:
                return eps, minSamples, labels

    return None, None, None


In [223]:
def evaluateKMeansFamily():

    results = kmeansApproach(features2D, K=3)


    for name, labels in results.items():

        fmScore = evaluateClustering(predLabels=labels, trueLabels=allLabels, method="fowlkes")
        scScore = evaluateClustering(predLabels=labels, features=features2D, method="silhouette")


        print(name, "→ Fowlkes-Mallows:", round(fmScore, 3), "Silhouette:", round(scScore, 3))





In [224]:
def evaluateSpectralClustering(spectralLabels, allLabels, features2D):
    fmScore = evaluateClustering(predLabels=spectralLabels, trueLabels=allLabels, method="fowlkes")
    scScore = evaluateClustering(predLabels=spectralLabels, features=features2D, method="silhouette")

    print("SpectralClustering → Fowlkes-Mallows:", round(fmScore, 3), "Silhouette:", round(scScore, 3))


In [225]:
def evaluateAgglomerativeClustering():
    # Compute labels for all linkage methods
    aggloLabels = agglomerativeClusteringAll(features2D, n_clusters=3)  #(Integers)

    # Evaluate each linkage method
    for linkage, labels in aggloLabels.items():
        fmScore = evaluateClustering(predLabels=labels, trueLabels=allLabels, method="fowlkes")
        scScore = evaluateClustering(predLabels=labels, features=features2D, method="silhouette")

        print("Agglomerative (", linkage, ") → Fowlkes-Mallows:", round(fmScore, 3), "Silhouette:", round(scScore, 3))


In [226]:
def evaluateDBSCAN(features2D, allLabels):

    eps, minSamples, labels = searchClusterSize(features2D, desired_clusters=3)

    if eps is None:
        print("DBSCAN could not find 3 clusters.")
        return None, None, None, None, None

    fmScore = evaluateClustering(predLabels=labels, trueLabels=allLabels, method="fowlkes")
    scScore = evaluateClustering(predLabels=labels, features=features2D, method="silhouette")

    return eps, minSamples, fmScore, scScore, labels


In [227]:


# 1 Load dataset
imageFolder = "/content/drive/MyDrive/DataSets/Covid/Covid19-dataset/allImages"
dataset, loader = loadImages(imageFolder)

# 2 Transforms
transform = getTransform()

# 3 Device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 4 Feature extractor
featureExtractor, device = getResnet18FeatureExtractor()

# 5 Extract features
allFeatures, allLabels = extractFeatures(loader, featureExtractor, device)

# 6 PCA to 2D
features2D = reduceTo2D(allFeatures.numpy())

#7. Call Kmean Family
kmeanFamily=kmeansApproach(features2D)
print(kmeanFamily)


#9. Spectral Clustering
spectralLabels = spectralClusteringApproach(features2D)
print("Spectral clustering labels:", spectralLabels)

#10 Find parameter to get 3 clusters:
eps,minSamples,labels=searchClusterSize(features2D, 3)
print("Eps and min samples\t",eps,"\t",minSamples)




Sample image shape after transform: torch.Size([3, 224, 224])


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Original feature dim: 25088
Reduced feature dim: 2
{'KMeans_random': array([2, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2,
       0, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 0, 2, 2,
       0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0,
       1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 

In [228]:
evaluateKMeansFamily()
evaluateAgglomerativeClustering()
spectralLabels = spectralClusteringApproach(features2D)
evaluateSpectralClustering(spectralLabels, allLabels, features2D)



KMeans_random → Fowlkes-Mallows: 0.679 Silhouette: 0.548
KMeans_kmeans++ → Fowlkes-Mallows: 0.679 Silhouette: 0.548
Bisecting_KMeans → Fowlkes-Mallows: 0.763 Silhouette: 0.512
Agglomerative ( single ) → Fowlkes-Mallows: 0.578 Silhouette: -0.273
Agglomerative ( complete ) → Fowlkes-Mallows: 0.577 Silhouette: 0.492
Agglomerative ( average ) → Fowlkes-Mallows: 0.634 Silhouette: 0.533
Agglomerative ( ward ) → Fowlkes-Mallows: 0.621 Silhouette: 0.516
SpectralClustering → Fowlkes-Mallows: 0.688 Silhouette: 0.544


In [229]:
eps, minSamples, fm, sc, db_labels = evaluateDBSCAN(features2D, allLabels)

if eps is not None:
    print("=== DBSCAN Evaluation ===")
    print("eps:", eps)
    print("min_samples:", minSamples)
    print("Fowlkes-Mallows:", round(fm, 3))
    print("Silhouette Score:", round(sc, 3))
else:
    print("DBSCAN could not produce 3 clusters with provided grid.")


=== DBSCAN Evaluation ===
eps: 0.2
min_samples: 4
Fowlkes-Mallows: 0.554
Silhouette Score: -0.086


In [230]:
#Not necessary, but illustrates the search
eps_values = [0.1, 0.15, 0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5, 0.6, 0.7, 0.8, 0.9]
min_samples_values = [2,3,4,5,6]


for eps in eps_values:
    for minSamples in min_samples_values:
        clusterCount, labels = testDbscanPair(features2D, eps, minSamples)
        print(f"eps={eps}, minSamples={minSamples} → clusters={clusterCount}")

eps=0.1, minSamples=2 → clusters=19
eps=0.1, minSamples=3 → clusters=0
eps=0.1, minSamples=4 → clusters=0
eps=0.1, minSamples=5 → clusters=0
eps=0.1, minSamples=6 → clusters=0
eps=0.15, minSamples=2 → clusters=27
eps=0.15, minSamples=3 → clusters=1
eps=0.15, minSamples=4 → clusters=1
eps=0.15, minSamples=5 → clusters=1
eps=0.15, minSamples=6 → clusters=0
eps=0.2, minSamples=2 → clusters=44
eps=0.2, minSamples=3 → clusters=7
eps=0.2, minSamples=4 → clusters=3
eps=0.2, minSamples=5 → clusters=1
eps=0.2, minSamples=6 → clusters=0
eps=0.25, minSamples=2 → clusters=50
eps=0.25, minSamples=3 → clusters=10
eps=0.25, minSamples=4 → clusters=4
eps=0.25, minSamples=5 → clusters=1
eps=0.25, minSamples=6 → clusters=0
eps=0.3, minSamples=2 → clusters=57
eps=0.3, minSamples=3 → clusters=13
eps=0.3, minSamples=4 → clusters=7
eps=0.3, minSamples=5 → clusters=2
eps=0.3, minSamples=6 → clusters=0
eps=0.35, minSamples=2 → clusters=58
eps=0.35, minSamples=3 → clusters=24
eps=0.35, minSamples=4 → clusters=